In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.request import Request, urlopen
import requests
from bs4 import BeautifulSoup as soup
import re

https://medium.com/@raiyanquaium/how-to-web-scrape-using-beautiful-soup-in-python-without-running-into-http-error-403-554875e5abed

In [4]:
url1 = 'https://www.metacritic.com/game/pc/doom-ii'
req = Request(url1, headers={'User-Agent': 'Mozilla/5.0'})
webpage1 = urlopen(req).read()


In [63]:
url2 = 'https://www.metacritic.com/game/pc/full-throttle'
req = Request(url2, headers={'User-Agent': 'Mozilla/5.0'})
webpage2 = urlopen(req).read()

url3 = 'https://www.metacritic.com/game/pc/big-rigs-over-the-road-racing'
req = Request(url3, headers={'User-Agent': 'Mozilla/5.0'})
webpage3 = urlopen(req).read()
soup3 = soup(webpage3,"html.parser")

In [6]:
soup1 = soup(webpage1,"html.parser")
soup2 = soup(webpage2,"html.parser")
print(soup1.prettify())


<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
   "https://www.w3.org/TR/html4/strict.dtd">
<html xml:lang="en">
 <head>
  <title>
   DOOM II for PC Reviews - Metacritic
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <script type="text/javascript">
   window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o||n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({1:[function(e,n,t){function r(){}function o(e,n,t){return function(){return i(e,[c.now()].concat(u(arguments)),n?null:this,t),n?void 0:this}}var i=e("handle"),a=e(3),u=e(4),f=e("ee").get("tracer"),c=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var p=["setPageViewName","setCustomAttribute","setErrorHandler","finished","addToTrace","inlineHit","addRelease"],d="api-",l=d+"ixn-";a

In [7]:
print(soup2.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
   "https://www.w3.org/TR/html4/strict.dtd">
<html xml:lang="en">
 <head>
  <title>
   Full Throttle for PC Reviews - Metacritic
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <script type="text/javascript">
   window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o||n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({1:[function(e,n,t){function r(){}function o(e,n,t){return function(){return i(e,[c.now()].concat(u(arguments)),n?null:this,t),n?void 0:this}}var i=e("handle"),a=e(3),u=e(4),f=e("ee").get("tracer"),c=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var p=["setPageViewName","setCustomAttribute","setErrorHandler","finished","addToTrace","inlineHit","addRelease"],d="api-",l=d+"i

In [8]:
critic_critique = soup2.find(itemprop="ratingValue").getText()
print(critic_critique)

86


In [9]:
critic_critique = soup1.find(itemprop="ratingValue").getText()
print(critic_critique)

83


In [10]:
test = "here is a test where I am trying to find metascore_w"
find_meta = re.compile(r'metascore_w')
find_anything = re.compile('.')
public_score = soup1.find_all(class_=find_meta)[2].getText()
print(public_score)

8.9


In [11]:
public_score = soup2.find_all(class_=find_meta)[2].getText()
print(public_score)

8.8


In [12]:
find_critic_rev_num = re.compile(r'\/game\/pc\/.*\/critic-reviews')
critic_rev_num = soup1.find_all(href=find_critic_rev_num)[2].getText().replace("\n","").replace(" ","").replace("Critics","")
print(critic_rev_num)

5


In [13]:
critic_rev_num = soup2.find_all(href=find_critic_rev_num)[2].getText().replace("\n","").replace(" ","").replace("Critics","")
print(critic_rev_num)

8


In [14]:
find_user_rev_num = re.compile(r'\/game\/pc\/.*\/user-reviews')
user_rev_num = soup1.find_all(href=find_user_rev_num)[2].getText().replace(' Ratings', "")
print(user_rev_num)

366


In [15]:
user_rev_num = soup2.find_all(href=find_user_rev_num)[2].getText().replace(' Ratings', "")
print(user_rev_num)

160


In [68]:
find_genre = re.compile("Action|Adventure|Sci-Fi|Shooter|First-Person|Arcade") #Expand on library for genres
genres=[]
for row in soup1.find_all('span', class_="data"):
    text = row.getText()
    if re.search(find_genre,text):
        genres.append(re.search(find_genre,text).group())

print(set(genres))

{'Shooter', 'Sci-Fi', 'First-Person', 'Action', 'Arcade'}


In [88]:
soup2.find_all('span', class_="data")
labels = soup2.find_all('span', class_="label")
for label in labels:
    if 'Genre' in label.text:
        sibs = label.next_siblings

list(sibs)

[<span class="data">Adventure</span>,
 ',                                            ',
 <span class="data">General</span>,
 ' ']

In [58]:
genres=[]
for row in soup2.find_all('span', class_="data"):
    text = row.getText()
    if re.search(find_genre,text):
        genres.append(re.search(find_genre,text).group())
print(set(genres))

<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
{'Adventure'}


In [61]:
find_esrb = re.compile('EC|E|E10+|T|M|AO|RP')
for row in soup1.find_all('span', class_="data"):
    text = row.getText()
    if re.search(find_esrb,text):
        print(re.search(find_esrb,text))

<re.Match object; span=(59, 60), match='T'>
<re.Match object; span=(33, 34), match='T'>
<re.Match object; span=(0, 1), match='M'>


In [62]:
find_esrb = re.compile('EC|E|E10+|T|M|AO|RP')
for row in soup2.find_all('span', class_="data"):
    text = row.getText()
    if re.search(find_esrb,text):
        print(re.search(find_esrb,text))

<re.Match object; span=(191, 192), match='T'>


In [75]:
find_esrb = re.compile('EC?|E?|E10+?|T?|M?|AO?|RP?')
for row in soup3.find_all('span', class_="data"):
    text = row.getText()
    matches = re.search(find_esrb,text)
    match = matches[0]
    if match:
        print(match)
#     print(repr(matches[0\/]))


E
